Imports

In [29]:
# Import necessari
import cv2
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import json
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

Canny edge detection

In [20]:
def get_edge_image(image, low_threshold, high_threshold):
    # Convert the image to grayscale if it's not already
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detection
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    
    return edges

# Load an image
image = cv2.imread(r'C:\Users\loren\Downloads\APKLOT-master\APKLOT-master\1. Satellite\Dataset\parcheggi\Dataset\402492112_training.png')

# Get the edge image
edge_image = get_edge_image(image, low_threshold=250, high_threshold=350)

# Display the edge image
cv2.imshow('Edge Image', edge_image)
cv2.waitKey(0)

-1

Store canny edge

In [ ]:
dataset_path = r'C:\Users\loren\Downloads\APKLOT-master\APKLOT-master\1. Satellite\Dataset\parcheggi\Dataset'
edge_path = r'C:\Users\loren\Downloads\APKLOT-master\APKLOT-master\1. Satellite\Dataset\parcheggi\Dataset_edge'

for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        input_image_path = os.path.join(dataset_path, filename)
        image = cv2.imread(input_image_path)
        # Ottieni l'immagine del bordo
        edge_image = get_edge_image(image, low_threshold=250, high_threshold=350)
        # Salva l'immagine del bordo nella cartella di output
        output_image_path = os.path.join(edge_path, filename)
        cv2.imwrite(output_image_path, edge_image)

print("Il rilevamento dei bordi è stato applicato a tutte le immagini nella cartella.")

Dataset preparation

In [52]:
# Custom dataset class
class ParkingLotDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transforms=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transforms = transforms

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]

        # Load image
        image = cv2.imread(image_path)
        #image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load masks
        with open(mask_path, 'r') as f:
            masks_data = json.load(f)

        masks = []
        for shape in masks_data['shapes']:
            points = np.array(shape['points'])
            mask = np.zeros_like(image[:, :, 0])
            cv2.fillPoly(mask, [points.astype(np.int32)], 1)
            masks.append(mask)

        masks = np.stack(masks, axis=0).astype(np.float32)

        # Apply transformations
        if self.transforms:
            image = self.transforms(image)
            masks = self.transforms(masks)

        return image, masks

# Initialize variables to store max height and width
max_height = 0
max_width = 0

# Iterate through all images in the directory
for filename in os.listdir('C:/Users/loren/Downloads/APKLOT-master/APKLOT-master/1. Satellite/Dataset/parcheggi/dataloader'):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join('C:/Users/loren/Downloads/APKLOT-master/APKLOT-master/1. Satellite/Dataset/parcheggi/dataloader', filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # Update max_height and max_width if necessary
        if height > max_height:
            max_height = height
        if width > max_width:
            max_width = width

print("Maximum Height:", max_height)
print("Maximum Width:", max_width)

# Data transformations
transform = transforms.Compose([
    transforms.Resize((max_height, max_width)),  # Resize to maximum dimensions
    transforms.Pad(padding),  # Pad images to make them all the same size
    transforms.ToTensor()
])

# Combine image and mask paths
image_paths = []
mask_paths = []
for filename in os.listdir('C:/Users/loren/Downloads/APKLOT-master/APKLOT-master/1. Satellite/Dataset/parcheggi/dataloader'):
    if filename.endswith('.png'):
        image_path = 'C:/Users/loren/Downloads/APKLOT-master/APKLOT-master/1. Satellite/Dataset/parcheggi/dataloader'+ '/' + filename
        mask_path = 'C:/Users/loren/Downloads/APKLOT-master/APKLOT-master/1. Satellite/Dataset/parcheggi/dataloader'+ '/' + filename.replace('.png', '.json')
        image_paths.append(image_path)
        mask_paths.append(mask_path)

# Create dataset
dataset = ParkingLotDataset(image_paths, mask_paths, transforms=transform)

# Split image and mask paths into train and test sets
train_images, test_images, train_masks, test_masks = train_test_split(dataset.image_paths, dataset.mask_paths, test_size=0.2, random_state=42)

# Further split train set into train and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(train_images, train_masks, test_size=0.2, random_state=42)

# Define datasets and data loaders for train, validation, and test sets
train_dataset = ParkingLotDataset(train_images, train_masks, transforms=transform)
val_dataset = ParkingLotDataset(val_images, val_masks, transforms=transform)
test_dataset = ParkingLotDataset(test_images, test_masks, transforms=transform)

# Create data loaders
train_data_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)

import matplotlib.pyplot as plt

# Display images and masks from the train data loader
for images, masks in train_data_loader:
    for i in range(images.size(0)):
        image = images[i].permute(1, 2, 0).numpy()
        mask = masks[i].permute(1, 2, 0).numpy()

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        ax1.imshow(image)
        ax1.set_title('Image')
        ax2.imshow(mask.squeeze(), cmap='gray')
        ax2.set_title('Mask')
        plt.show()
        break  # Display only one batch
    break  # Display only one batch

Maximum Height: 2108
Maximum Width: 2113


NameError: name 'padding' is not defined

In [51]:
import cv2
import os

# Directory containing the images
data_dir = 'C:/Users/loren/Downloads/APKLOT-master/APKLOT-master/1. Satellite/Dataset/parcheggi/dataloader'

# Initialize variables to store max height and width
max_height = 0
max_width = 0

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # Update max_height and max_width if necessary
        if height > max_height:
            max_height = height
        if width > max_width:
            max_width = width

print("Maximum Height:", max_height)
print("Maximum Width:", max_width)

Maximum Height: 2108
Maximum Width: 2113


Model

In [ ]:
# Load pre-trained Mask R-CNN model
model = maskrcnn_resnet50_fpn(pretrained=True)

# Set the number of classes (in this case, 1 for parking lot regions)
num_classes = 2  # Background and parking lot region

# Get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-trained classifier with a new one
model.roi_heads.box_predictor = maskrcnn_resnet50_fpn.FastRCNNPredictor(in_features, num_classes)

# Move the model to the desired device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the loss functions
criterion = nn.CrossEntropyLoss()
maskcriterion = nn.BCEWithLogitsLoss()

# Define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

Train